In [121]:
%matplotlib inline
import pandas as pd
import bs4
from bs4 import BeautifulSoup
import numpy as np
import re
import matplotlib.pyplot as plt
import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest
import math

In [37]:
df = pd.read_csv("FullData.csv", header=0)
df_capes = pd.read_csv("capeReviewsCleaned2.csv", header=0)
df_rmp = pd.read_csv("modifiedProfInfo.csv")

In [14]:
df_test_222495 = df_capes[(df_capes['tid'] == 222495) & pd.notnull(df_capes['Avg Grade Received'])]

In [29]:
tot_evals = df_test_222495['Evals Made'].values.sum()

In [31]:
w_gpa = 0
for index, col in df_test_222495.iterrows():
    w_eval = col['Evals Made'] / tot_evals
    w_gpa += w_eval * col['Avg Grade Received']
w_gpa    

3.338421052631579

In [33]:
df[df['tid'] == 222495]

,tid,gender,tDept,Enroll,Evals Made,Eval/Enroll,Rcmnd Class,Rcmnd Instr,Study Hrs/wk,Avg Grade Expected,Avg Grade Received,rEasy,rHelpful,rInterest,rOverall,rWouldTakeAgain,rmp Grade,sentimentValue,teacherRatingTags
125,222495,1.0,ANTH,1049,522,0.497617,0.815161,0.832031,4.088793,3.407922,3.338421,2.515152,3.333333,3.185185,3.378788,0.6,3.666667,0.066216,NaN


In [36]:
df.head()

,tid,gender,tDept,Enroll,Evals Made,Eval/Enroll,Rcmnd Class,Rcmnd Instr,Study Hrs/wk,Avg Grade Expected,Avg Grade Received,rEasy,rHelpful,rInterest,rOverall,rWouldTakeAgain,rmp Grade,sentimentValue,teacherRatingTags
0,7853,1.0,PHYS,3803,1478,0.388641,0.717524,0.531486,6.288403,3.100710,2.898160,3.217949,2.461538,3.116279,2.474359,0.333333,3.480000,0.068542,NaN
1,7855,1.0,CHEM,19349,9774,0.505142,0.861172,0.885933,7.689782,3.014609,2.860434,3.716535,3.916010,3.238095,4.018373,0.730769,3.142857,0.140177,NaN
2,7857,1.0,MMW,2529,1579,0.624357,0.846111,0.947168,6.370589,3.312166,3.067663,2.883721,4.534884,3.508772,4.575581,NaN,3.000000,0.287243,NaN
3,7858,1.0,MAE,3328,1333,0.400541,0.845998,0.716439,6.840503,3.414405,3.110601,2.864407,3.169492,3.228571,2.711864,0.000000,4.000000,0.031384,NaN
4,7859,1.0,MATH,1958,851,0.434627,0.791921,0.683295,6.483631,3.285734,2.728183,2.829268,2.536585,3.166667,2.390244,0.000000,NaN,0.064891,NaN


In [131]:
df_tids = df['tid'].values
df_rmp_tids = df_rmp['tid'].values
df_rmp_tids = [value for value in df_rmp_tids if not 'NaN']

In [132]:
df_set = np.setdiff1d(df_tids,df_rmp_tids )
df_set = df_set.astype(int)

In [173]:
ee = []
for tid in df_set:
    eval_enroll = len(df_rmp[df_rmp['tid'] == str(tid)]) / df[df['tid'] == tid]['Enroll'].values[0]
    ee.append(eval_enroll)

In [175]:
df['Eval/Enroll RMP'] = ee

In [200]:
def getRatingTags(tid):
    tags = []
    for index, row in rmp[rmp['tid'] == tid].iterrows():
        tg = row['teacherRatingTags']
        if(not pd.isnull(tg)):
            tg = tg.strip('[ ]')
            tg = tg.replace('\'', ' ')
            tg = tg.split(',')
            tags.extend([a.strip() for a in tg if (a.strip() not in tags)])
    if('' in tags):    
        tags.remove('')
        
    tgstr = ""
    for tg in tags:
        tgstr += tg
        tgstr += ','
        
    return tgstr

In [197]:
rmp = pd.read_csv("modifiedProfInfo.csv")

In [223]:
a = getRatingTags('2325936')
print(a)
type(df_set[0])

Lecture heavy,Caring,Lots of homework,Hilarious,Accessible outside class,Amazing lectures,Inspirational,


numpy.int64

In [225]:
for tid in df_set:
    index = new_df[new_df['tid'] == tid].index
    new_df.loc[index, 'teacherRatingTags'] = getRatingTags(str(tid))

['Get ready to read', 'Test heavy', 'Tough grader', '']

In [209]:
new_df = pd.read_csv("FullData.csv", header=0)

In [226]:
new_df.head()

,tid,gender,tDept,Enroll,Evals Made,Eval/Enroll,Rcmnd Class,Rcmnd Instr,Study Hrs/wk,Avg Grade Expected,Avg Grade Received,rEasy,rHelpful,rInterest,rOverall,rWouldTakeAgain,rmp Grade,sentimentValue,teacherRatingTags,rmp Evals/Enroll
0,7853,1.0,PHYS,3803,1478,0.388641,0.717524,0.531486,6.288403,3.100710,2.898160,3.217949,2.461538,3.116279,2.474359,0.333333,3.480000,0.068542,"Get ready to read,Test heavy,Tough grader,",0.020510
1,7855,1.0,CHEM,19349,9774,0.505142,0.861172,0.885933,7.689782,3.014609,2.860434,3.716535,3.916010,3.238095,4.018373,0.730769,3.142857,0.140177,"Amazing lectures,Caring,Tough grader,Respected...",0.019691
2,7857,1.0,MMW,2529,1579,0.624357,0.846111,0.947168,6.370589,3.312166,3.067663,2.883721,4.534884,3.508772,4.575581,NaN,3.000000,0.287243,,0.034006
3,7858,1.0,MAE,3328,1333,0.400541,0.845998,0.716439,6.840503,3.414405,3.110601,2.864407,3.169492,3.228571,2.711864,0.000000,4.000000,0.031384,"Clear grading criteria,Graded by few things,Lo...",0.017728
4,7859,1.0,MATH,1958,851,0.434627,0.791921,0.683295,6.483631,3.285734,2.728183,2.829268,2.536585,3.166667,2.390244,0.000000,NaN,0.064891,"Lecture heavy,Lots of homework,""Skip class? Yo...",0.020940


In [211]:
new_df['teacherRatingTags'] = strarr

,tid,gender,tDept,Enroll,Evals Made,Eval/Enroll,Rcmnd Class,Rcmnd Instr,Study Hrs/wk,Avg Grade Expected,Avg Grade Received,rEasy,rHelpful,rInterest,rOverall,rWouldTakeAgain,rmp Grade,sentimentValue,teacherRatingTags,rmp Evals/Enroll
0,7853,1.0,PHYS,3803,1478,0.388641,0.717524,0.531486,6.288403,3.100710,2.898160,3.217949,2.461538,3.116279,2.474359,0.333333,3.480000,0.068542,"Get ready to read,Test heavy,Tough grader,",0.020510
1,7855,1.0,CHEM,19349,9774,0.505142,0.861172,0.885933,7.689782,3.014609,2.860434,3.716535,3.916010,3.238095,4.018373,0.730769,3.142857,0.140177,"Amazing lectures,Caring,Tough grader,Respected...",0.019691
2,7857,1.0,MMW,2529,1579,0.624357,0.846111,0.947168,6.370589,3.312166,3.067663,2.883721,4.534884,3.508772,4.575581,NaN,3.000000,0.287243,,0.034006
3,7858,1.0,MAE,3328,1333,0.400541,0.845998,0.716439,6.840503,3.414405,3.110601,2.864407,3.169492,3.228571,2.711864,0.000000,4.000000,0.031384,"Clear grading criteria,Graded by few things,Lo...",0.017728
4,7859,1.0,MATH,1958,851,0.434627,0.791921,0.683295,6.483631,3.285734,2.728183,2.829268,2.536585,3.166667,2.390244,0.000000,NaN,0.064891,"Lecture heavy,Lots of homework,""Skip class? Yo...",0.020940
5,9709,1.0,HUM,3744,1712,0.457265,0.675003,0.790459,6.617617,2.924550,2.721314,3.275862,3.448276,2.652174,3.250000,0.000000,2.666667,-0.004256,"Lecture heavy,Get ready to read,Amazing lectur...",0.015491
6,26354,1.0,PSYC,121,85,0.702479,0.835000,0.918000,3.630000,NaN,NaN,3.945946,2.972973,3.714286,3.175676,0.000000,2.000000,0.069811,"Tough grader,",0.305785
7,29553,1.0,ICAM,988,519,0.525304,0.707013,0.697555,4.291098,3.558410,3.184041,3.272727,1.727273,2.733333,1.704545,NaN,NaN,-0.079896,,0.022267
8,32593,1.0,POLI,1308,519,0.396789,0.812904,0.763243,4.915299,3.344759,2.902832,2.500000,3.647059,3.629630,3.985294,NaN,NaN,0.192362,,0.025994
9,35989,1.0,MMW,2535,1484,0.585404,0.728631,0.879964,6.208181,3.285121,3.155860,2.685185,4.018519,3.210526,3.898148,0.750000,3.900000,0.251128,"Beware of pop quizzes,Accessible outside class...",0.021302


In [229]:
new_df.to_csv("FullData.csv", index=False)